# Vehicle Lane Positioning in CARLA

In [1]:
import carla
import random

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()
map = world.get_map()
tm = client.get_trafficmanager()

In [3]:
vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
spawn_points = map.get_spawn_points()
spectator = world.get_spectator()

## Measuring Vehicle and Lane Dimensions

Create vehicle

In [4]:
ego_vehicle = world.spawn_actor(random.choice(vehicle_bps), random.choice(spawn_points))

Move spectator camera behind the vehicle

In [5]:
spectator_transform = ego_vehicle.get_transform()
spectator_transform.location += -8 * ego_vehicle.get_transform().get_forward_vector()
spectator_transform.location.z += 3
spectator_transform.rotation.pitch = -15
spectator.set_transform(spectator_transform)

Get details about vehicle location, width, and width of the lane it is in

In [6]:
print(ego_vehicle.get_location())
print(ego_vehicle.bounding_box.extent)
ego_vehicle_width = ego_vehicle.bounding_box.extent.y * 2
ego_vehicle_width

Location(x=-18.385923, y=130.210068, z=0.590864)
Vector3D(x=2.305503, y=1.120857, z=0.833638)


2.241713285446167

In [7]:
ego_vehicle_waypoint = map.get_waypoint(ego_vehicle.get_location())
ego_vehicle_waypoint.lane_width

3.5

Draw the vehicle bounding box and then use the vehicle width and lane width to draw a larger bounding box around the vehicle that fills the whole lane width.

In [8]:
# Draw the bounding box of the ego vehicle
transform = ego_vehicle.get_transform()
bounding_box = ego_vehicle.bounding_box
bounding_box.location += transform.location
world.debug.draw_box(bounding_box, transform.rotation, 0.1, carla.Color(32, 0, 0, 0), 15)

# Draw the expanded bounding box of the ego vehicle out to the edges of the lane (expand y axis of the bounding box)
remaining_lane_width = ego_vehicle_waypoint.lane_width - ego_vehicle_width
bounding_box.extent.y += remaining_lane_width / 2 # Extend to the edges of the lane
bounding_box.extent.x += 0.3 # For better visualization
bounding_box.extent.z += 0.3 # For better visualization
bounding_box.location.z += 0.3 # For better visualization
world.debug.draw_box(bounding_box, transform.rotation, 0.1, carla.Color(0, 32, 0, 0), 15)

Determine how exactly the lane offset impacts the vehicle's position in the world

In [9]:
ego_vehicle.set_autopilot(True)

In [10]:
print(ego_vehicle.get_location())

Location(x=-18.385923, y=130.210068, z=0.422425)


In [11]:
tm.vehicle_lane_offset(ego_vehicle, -1.75)

In [12]:
print(ego_vehicle.get_location())

Location(x=-18.385923, y=130.210068, z=0.307803)


**Conclusion:** The lane offset is in meters, not some percentage of the lane width. Setting this to + or - the width of a lane will move the car to the center of the adjacent lane (though it will throw off the planning algorithm). Setting it to + or - half the lane with will make the car drive right on the road line.

## Normalized Lane-and-Vehicle-Width-Independent Representation of Lane Positions

Normalized lane-and-vehcile-width-independent positions represent a vehicle's position in a lane proportionally to the empty space on both sides of a vehicle within the lane it is travelling in. This normalized lane offset value expresses this in terms of 1/2 of the lane width, excluding the width of the vehicle in the lane `(0.5 * (lane_width - vehicle_width))`, with the sign indicating if the vehicle is offset towards the left or right line of the lane. This describes a vehicle lane position in a lane-and-vehicle-width-independent way, where the values of 1 and -1 represent the vehicle driving with its right and left edge/side against the right and left road line, respectively, and 0 represents the vehicle driving in the center of the lane.

The utility of this representation comes in that it describes the position within a lane that a driver drives in relative to the available space in the lane, which varies based on the size of the vehicle, according to typical driving behavior that tries to always stay in their lane. (Ex. a semi truck will generally stay centered as they often do not have much horizontal space in a lane to manuver within, compared to a sedan). Using a lane-size-independent representation that does not account for the width of the vehicle (ex. normalized vehicle center offset) can result in unintended overlap into adjacent lanes if applying such an offset to a vehicle with a width other than the vehicle width used as reference when determining the offset (whether arbitrary or recorded from actual data). The lane-and-vehicle-width-independent representation of lane offsets is therefore more general, though it does give up a bit of direct control over certain aspects without extra computation, compared to absolute or center-based offsets that do not adjust based on vehicle width. Aspects such as defining a specific number of meters of overlap into adjacent lanes, which would be better served by absolute offsets, is one such example where this representation is not the best choice. For the charging efficiency based on lane position for wireless charging in-road, this representation is more appropriate as it describes the vehicle's position in the lane relative to the available space in the lane, mirroring typical driving behavior.

Accurately simulating a real power consumption of wireless chargers is also dependent on simulating accurately the distribution of vehicle types on the road, as the width of a vehicle does have impact on where a driver drives in a lane as larger vehicles have less space in a lane as they take up more of it. This is generally applicable regardless of the lane offset representation type used. For this representation, it helps reduce variability. This representation is also more useful in simulation environments where a vehicle may travel through lanes of different widths.

### Interpreting Lane Position Values:

- 0: Vehicle is in center of the lane
- -1: Vehicle drives with its left edge/side against the left road line
- 1: Vehicle drives with its right edge/side against the right road line
- Values greater than or smaller than -1 or 1 indicate that the vehicle is overlapping/encroaching into the adjacent lane.
- Values between 1 and -1, excluding 0, indicate that the vehicle is driving in the lane but not in the center.


In [13]:
def lane_position_to_center_offset_meters(offset_normalized: float, vehicle):
    waypoint = map.get_waypoint(vehicle.get_location())

    half_lane_width = waypoint.lane_width / 2
    half_vehicle_width = vehicle.bounding_box.extent.y
    half_open_lane_space = half_lane_width - half_vehicle_width
    
    if half_open_lane_space <= 0:
        # Normalized lane-and-vehicle-width-independent-offset representation is not
        # intended to model lane positioning of oversize vehicles that are wider than or as wide as the lane width.
        # For our purposes, we are assuming that oversize vehicles will attempt to remain as centered
        # in the lane as possible.
        return 0
    
    # Sign is only for determining left or right lane offset. Computation must use the absolute value.
    normalized_offset_abs = abs(offset_normalized)
    
    # Convert normalized lane offset to vehicle center offset in meters
    carla_offset_meters = half_lane_width - (half_open_lane_space - normalized_offset_abs * half_open_lane_space) - half_vehicle_width
    
    # Carry over the original sign so that we have a CARLA vehicle_line_offset value    
    return carla_offset_meters if offset_normalized >= 0 else -carla_offset_meters

In [14]:
def center_offset_meters_to_lane_position(offset_meters: float, vehicle):
    waypoint = map.get_waypoint(vehicle.get_location())

    half_lane_width = waypoint.lane_width / 2
    half_vehicle_width = vehicle.bounding_box.extent.y
    half_open_lane_space = half_lane_width - half_vehicle_width
    
    if half_open_lane_space <= 0:
        # Normalized lane-and-vehicle-width-independent-offset representation is not
        # intended to model lane positioning of oversize vehicles that are wider than or as wide as the lane width.
        # For our purposes, we are assuming that oversize vehicles will attempt to remain as centered
        # in the lane as possible.
        return 0
    
    # Sign is only for determining left or right lane offset. Computation must use the absolute value.
    offset_meters_abs = abs(offset_meters)
    
    # Convert vehicle center offset in meters to normalized lane offset
    normalized_offset = (offset_meters_abs + half_vehicle_width + half_open_lane_space - half_lane_width) / half_open_lane_space
    
    # Carry over the original sign so that we have a normalized lane offset value
    return normalized_offset if offset_meters >= 0 else -normalized_offset

In [15]:
lane_offset = lane_position_to_center_offset_meters(-0.62, ego_vehicle)
print(lane_offset)
print(center_offset_meters_to_lane_position(lane_offset, ego_vehicle))
tm.vehicle_lane_offset(ego_vehicle, lane_offset)

-0.39006888151168817
-0.6199999999999996


This lane position representation is what we are using to model and implement vehicle lane position for our wireless charging efficiency simulation in CARLA.

## CARLA Clean-Up

Destroy all vehicles

In [16]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.destroy()